In [1]:
import time
import random
from math import ceil
from decimal import Decimal
from typing import List, Tuple

import random
from sympy import Symbol, Poly


# Use a large prime as the modulus for arithmetic operations
prime: int = 180252380737439
threshold: int = 5
secret1: int = 100
secret2: int = 100

In [2]:
# Client-side
# All parties use the same prime and same threshold
def sample_shamir_polynomial(zero_value: int) -> List[int]:
    coefs = [zero_value] + [prime-1 for _ in range(threshold-1)] #[random.randrange(prime) for _ in range(threshold - 1)]
    return coefs

In [3]:
# Client-side
def evaluate_at_point(coefs: List[int], point: int) -> int:
    result = 0
    for coef in reversed(coefs):
        result = (coef + point * result) % prime
    return result

In [4]:
# Server-side
def interpolate_at_point(points_values: List[Tuple[int, int]], query_x_axis: int) -> int:
    x_vals, y_vals = zip(*points_values)
    constants = lagrange_constants_for_point(x_vals, query_x_axis)
    return sum(ci * vi for ci, vi in zip(constants, y_vals)) % prime

In [5]:
# Server-side
def lagrange_constants_for_point(points: List[int], query_x_axis: int) -> List[int]:
    constants = [0] * len(points)
    for i in range(len(points)):
        xi = points[i]
        num = 1
        denum = 1
        for j in range(len(points)):
            if j != i:
                xj = points[j]
                num = (num * (xj - query_x_axis)) % prime
                denum = (denum * (xj - xi)) % prime
        constants[i] = (num * pow(denum, -1, prime)) % prime
    return constants

In [6]:
# Client-side
def shamir_share(secret: int, num_shares: int) -> List[Tuple[int, int]]:
    polynomial = sample_shamir_polynomial(secret)
    shares = [(i, evaluate_at_point(polynomial, i)) for i in range(1, num_shares + 1)]
    return shares, polynomial

In [16]:
shares1, coeff1 = shamir_share(secret1, 10)
shares2, coeff2 = shamir_share(secret2, 10)

print('secret 1')
print(f'shares: {shares1}')
print(f'coeff: {coeff1}')

print('\nsecret 2')
print(f'shares: {shares2}')
print(f'coeff: {coeff2}')

secret 1
shares: [(1, 96), (2, 70), (3, 180252380737419), (4, 180252380737199), (5, 180252380736759), (6, 180252380735985), (7, 180252380734739), (8, 180252380732859), (9, 180252380730159), (10, 180252380726429)]
coeff: [100, 180252380737438, 180252380737438, 180252380737438, 180252380737438]

secret 2
shares: [(1, 96), (2, 70), (3, 180252380737419), (4, 180252380737199), (5, 180252380736759), (6, 180252380735985), (7, 180252380734739), (8, 180252380732859), (9, 180252380730159), (10, 180252380726429)]
coeff: [100, 180252380737438, 180252380737438, 180252380737438, 180252380737438]


In [8]:
import numpy as np
from matplotlib import pyplot as plt

def polynomial(x, coeffs):
    o = len(coeffs)
    y = 0
    for i in range(o):
        y += coeffs[i]*x**i
    return y

x = np.linspace(0, 9, 10)

In [9]:
# Server-side
def shamir_add(x, y):
    return [ (i+1, (xi[1] + yi[1]) % prime) for i, (xi, yi) in enumerate(list(zip(x, y))) ]

In [10]:
added = shamir_add(shares1, shares2)

In [11]:
# Server-side
def shamir_reconstruct(shares: List[Tuple[int, int]], query_x_axis: int = 0) -> int:
    polynomial = [(p, v) for p, v in shares]
    secret = interpolate_at_point(polynomial, query_x_axis)
    return secret

In [12]:
print('points:')
added

points:


[(1, 192),
 (2, 140),
 (3, 180252380737399),
 (4, 180252380736959),
 (5, 180252380736079),
 (6, 180252380734531),
 (7, 180252380732039),
 (8, 180252380728279),
 (9, 180252380722879),
 (10, 180252380715419)]

In [13]:
for i in range(len(added)+1):
    print(f'x: {i}, y: {shamir_reconstruct(added, i)}')

x: 0, y: 200
x: 1, y: 192
x: 2, y: 140
x: 3, y: 180252380737399
x: 4, y: 180252380736959
x: 5, y: 180252380736079
x: 6, y: 180252380734531
x: 7, y: 180252380732039
x: 8, y: 180252380728279
x: 9, y: 180252380722879
x: 10, y: 180252380715419


In [14]:
print(f'secret: {shamir_reconstruct(added, 0)}')

secret: 200
